In [11]:
source("../Scripts/functions.R")
getExperiment <- function(datafile, metafile=NULL){
   data <- readData(datafile)
   print(data[1:5,1:5])
   experiment <- SingleCellExperiment::SingleCellExperiment(list(counts=data))
   if(!is.null(metafile)){
     meta <- read.csv(metafile)
     print(table(colnames(data)%in% meta$id))    
     rownames(meta) <- meta$id
     meta <- meta[colnames(data),]
     print(head(meta))                  
     print(all(colnames(data)== meta$id))
     SingleCellExperiment::colLabels(experiment) <- meta$class_
     experiment <- experiment[,!is.na(experiment$label)]  
     
   }
  experiment <- scater::logNormCounts(experiment)
  return(experiment)
}


Attaching package: ‘docstring’


The following object is masked from ‘package:utils’:

    ?




In [ ]:
data <- read.csv(datafile)
meta <- read.csv(metafile)

In [12]:
datafile <- "../Data/subsets/Mosaic_500_1/data_train.txt"
metafile <- "../Data/subsets/Mosaic_500_1/meta_train.txt"
train <- getExperiment(datafile, metafile)


Warning message in data.table::fread(data, sep = ",", verbose = F):
“Detected 4023 column names but the data has 4024 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


         pbmc1_Celseq2_1_TGCATG pbmc1_Celseq2_2_ACAGTG pbmc1_Celseq2_2_CTCATG
TSPAN6                        0                      0                      0
TNMD                          0                      0                      0
DPM1                          0                      0                      0
SCYL3                         0                      0                      0
C1orf112                      0                      0                      0
         pbmc1_Celseq2_2_GTACTC pbmc1_Celseq2_2_TGTCAC
TSPAN6                        0                      0
TNMD                          0                      0
DPM1                          0                      0
SCYL3                         0                      0
C1orf112                      0                      0

TRUE 
4023 
                                           id              class_     tech
pbmc1_Celseq2_1_TGCATG pbmc1_Celseq2_1_TGCATG         CD4+ T cell CEL-Seq2
pbmc1_Celseq2_2_ACAGTG pbmc1_Celseq2_2_

In [ ]:
test <- getExperiment("../Data/processed/PBMC_mono/data_test.txt")

In [ ]:
predictions<- as.data.frame(SingleR::SingleR(test=test, ref=train, labels=train$label, de.method="wilcox", de.n = 50))
#predictions<- as.data.frame(SingleR::SingleR(test=test, ref=train, labels=train$label, de.method="t", de.args=list(lfc=1)))
predictions$id <- rownames(predictions)
table(predictions$labels)

In [ ]:
meta_test <- read.csv("../Data/processed/PBMC_mono/meta_test.txt")
results <- merge(meta_test, predictions, by="id")
colnames(results)[colnames(results)== "labels"] <- "prediction"
results$prediction.match <- results$prediction == results$class_
(sum(results$prediction==results$class_, na.rm=T) / length(results$class_)
table(results$prediction.match)
#write.table(data, paste0(output,".txt") ,sep="\t", quote=F)

In [41]:
head(results)

,id,class_,scores.B.cell,scores.CD14..monocyte,scores.CD16..monocyte,scores.CD4..T.cell,scores.Cytotoxic.T.cell,scores.Dendritic.cell,scores.Megakaryocyte,scores.Natural.killer.cell,scores.Plasmacytoid.dendritic.cell,first.labels,tuning.scores.first,tuning.scores.second,prediction,pruned.labels,prediction.match
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<lgl>
1,pbmc1_Drop_AAAAACTGGGAT,Cytotoxic T cell,0.5889751,0.5195890,0.5222644,0.7429412,0.4710132,0.6315312,0.6629978,0.6047314,0.7439090,Plasmacytoid dendritic cell,0.4013292,0.3815603,Plasmacytoid dendritic cell,Plasmacytoid dendritic cell,FALSE
2,pbmc1_Drop_AAAACACATAAT,CD4+ T cell,0.6257914,0.5248090,0.5112893,0.8242623,0.4826509,0.5939835,0.7513957,0.6578693,0.7460584,CD4+ T cell,0.8242623,0.7513957,CD4+ T cell,CD4+ T cell,TRUE
3,pbmc1_Drop_AAAACCGATGAT,CD14+ monocyte,0.1916631,0.2471714,0.2314221,0.2000123,0.2295370,0.2033677,0.1639126,0.2465254,0.1986467,CD14+ monocyte,0.2006359,0.2002392,CD16+ monocyte,CD16+ monocyte,FALSE
4,pbmc1_Drop_AAAAGATGTGGT,CD4+ T cell,0.6040185,0.5495257,0.5594427,0.7823192,0.5052763,0.6382081,0.6862599,0.6375223,0.7514031,CD4+ T cell,0.4437179,0.1054180,CD4+ T cell,CD4+ T cell,TRUE
5,pbmc1_Drop_AAAAGGATTTCC,Cytotoxic T cell,0.5326199,0.5189248,0.5422052,0.6849021,0.4997567,0.6621688,0.6449290,0.5614334,0.7188406,Plasmacytoid dendritic cell,0.4244979,0.2051957,Plasmacytoid dendritic cell,Plasmacytoid dendritic cell,FALSE
6,pbmc1_Drop_AAAAGTGTTTAA,Cytotoxic T cell,0.5044643,0.4802225,0.4963272,0.6772935,0.4434499,0.6356188,0.5708002,0.5176660,0.7154570,Plasmacytoid dendritic cell,0.3102544,0.0000000,Plasmacytoid dendritic cell,Plasmacytoid dendritic cell,FALSE


In [44]:
colnames(results)[colnames(results)== "label"]

character(0)

In [13]:
file <- c("../Results/umap_PBMC.csv")
umap <- read.csv(file)
print(paste(nrow(umap), ncol(umap)))
umap <- umap[, stringr::str_detect(colnames(umap), "singleR", negate = T)] 
print(paste(nrow(umap), ncol(umap)))
umap$X <- NULL
umap$`X.1` <- NULL
umap[1:5,1:5]
write.csv(umap, file, row.names=F)

[1] "6584 1139"
[1] "6584 1139"


,id,Mono_Seurat_1000_18,Mono_Seurat_100_19,Mono_Seurat_100_20,Mono_Seurat_100_2
,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
1,pbmc1_Drop_AAAAACTGGGAT,TRUE,FALSE,FALSE,FALSE
2,pbmc1_Drop_AAAACACATAAT,TRUE,TRUE,TRUE,TRUE
3,pbmc1_Drop_AAAACCGATGAT,FALSE,FALSE,FALSE,FALSE
4,pbmc1_Drop_AAAAGATGTGGT,TRUE,TRUE,TRUE,FALSE
5,pbmc1_Drop_AAAAGGATTTCC,TRUE,FALSE,FALSE,FALSE


In [14]:
file <- c("../Results/umap_CrossSpecies.csv")
umap <- read.csv(file)
print(paste(nrow(umap), ncol(umap)))
umap <- umap[, stringr::str_detect(colnames(umap), "singleR", negate = T)] 
print(paste(nrow(umap), ncol(umap)))
umap$X <- NULL
umap$`X.1` <- NULL
umap[1:5,1:5]
write.csv(umap, file,row.names=F)

[1] "15693 601"
[1] "15693 601"


,id,CrossSpecies_Seurat_1000_10,CrossSpecies_Seurat_1000_11,CrossSpecies_Seurat_1000_12,CrossSpecies_Seurat_1000_13
,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
1,AAACCTGAGAAGGTTT.1_H3_normal,TRUE,TRUE,TRUE,TRUE
2,AAACCTGAGCAGCGTA.1_H4_normal,TRUE,TRUE,TRUE,TRUE
3,AAACCTGAGCCACGCT.1_H4_normal,TRUE,TRUE,TRUE,TRUE
4,AAACCTGAGGCGCTCT.1_H15T_normal,TRUE,TRUE,TRUE,TRUE
5,AAACCTGAGGTTCCTA.1_H15T_normal,TRUE,TRUE,FALSE,TRUE


In [15]:
file <- "../Results/celltypeAccuracies_PBMC.csv"
data <- read.csv(file, header=F)
data <- data[data$V1 != "singleR",]
head(data)
write.table(data, file,row.names=F, quote=F, col.names=F, sep=",")

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Seurat,Mono,1000,10,0.9807829,0.3725490,0.9141414,0.9078117,0.8518706,0.6521739,1,0.5392562,0.7500000,0.8485723
2,Seurat,Mono,1000,11,0.9829181,0.3669468,0.8232323,0.9053857,0.8700708,0.6521739,1,0.5557851,0.7142857,0.8517618
3,Seurat,Mono,1000,12,0.9857651,0.4005602,0.8636364,0.8859777,0.8675430,0.6086957,1,0.5185950,0.7500000,0.8456865
4,Seurat,Mono,1000,13,0.9843416,0.3305322,0.8989899,0.9019893,0.8751264,0.6304348,1,0.5516529,0.7142857,0.8523694
5,Seurat,Mono,1000,14,0.9850534,0.3333333,0.9040404,0.8947113,0.8725986,0.6739130,1,0.5495868,0.7142857,0.8499392
6,Seurat,Mono,1000,15,0.9822064,0.3809524,0.8383838,0.9107230,0.8806876,0.5652174,1,0.5847107,0.7857143,0.8595079


In [16]:
file <- "../Results/celltypeAccuracies_CrossSpecies.csv"
data <- read.csv(file, header=F)
data <- data[data$V1 != "singleR",]
head(data)
write.table(data, file,row.names=F, quote=F, col.names=F, sep=",")

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Seurat,CrossSpecies,1000,10,0.8967136,0.9707016,0.5770235,0.9814916,0.4283122,0.9136287,0.8356643,0.9361299,0.7850055,0.8766966
2,Seurat,CrossSpecies,1000,11,0.8873239,0.9622205,0.6344648,0.9804028,0.4773140,0.9141896,0.8531469,0.9023004,0.8088938,0.8767603
3,Seurat,CrossSpecies,1000,12,0.9061033,0.9683886,0.5770235,0.9809472,0.4718693,0.8925967,0.8678322,0.9263870,0.7952958,0.8757408
4,Seurat,CrossSpecies,1000,13,0.8967136,0.9699306,0.6214099,0.9809472,0.4246824,0.9242849,0.8594406,0.9334235,0.8195516,0.8874657
5,Seurat,CrossSpecies,1000,14,0.8967136,0.9614495,0.6005222,0.9798585,0.4555354,0.9223219,0.8615385,0.9361299,0.7666299,0.8784171
6,Seurat,CrossSpecies,1000,15,0.8638498,0.9714726,0.6266319,0.9809472,0.4410163,0.9054964,0.7993007,0.9328823,0.8636531,0.8856178


In [18]:
file <- "../Results/runtime.txt"
runtime <- read.table(file)
runtime <- runtime[runtime$V1 != "MLP",]
write.table(runtime, file,row.names=F, quote=F, col.names=F, sep=",")